# Load and preprocess 2004 data

We will, over time, look over other years. Our current goal is to explore the features of a single year.

---

In [1]:
%pylab --no-import-all inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [2]:
file = "../data/interim/2004data.dta"
df_rawest = pd.read_stata(file)

In [3]:
good_columns = [
    'V043116',  # Party Identification
    
    'V045132',  #Abortion
    'V045189',  #Moral Relativism
    'V045190',  #“Newer Lifestyles”
    'V045191',  #Moral Tolerance
    'V045192',  #Traditional Families
    'V045156a',  #Gay Job Discrimination
    'V045158',  # Gay adoption
    'V045157a',  # Gay military service
    
    'V043150',  #National Health Insurance
    'V043152',  #Guaranteed Job
    'V045121',  #Services/Spending
    
    'V045207a',  #Affirmative Action
    'V045193',  #Racial Resentment 1
    'V045194',  #Racial Resentment2
    'V045195',  #Racial Resentment3
    'V045196',  #Racial Resentment4
]
df_raw = df_rawest[good_columns]

## Clean the data
---

In [4]:
def convert_to_int(s):
    """Turn ANES data entry into an integer.
    
    >>> convert_to_int("1. Govt should provide many fewer services")
    1
    >>> convert_to_int("2")
    2
    """
    try:
        return int(s.partition('.')[0])
    except ValueError:
        warnings.warn("Couldn't convert: "+s)
        return np.nan
    except AttributeError:
        return s
    
def not_informative_to_nan(x):
    """Convert non-informative values to missing.
    
    ANES codes various non-answers as 8, 9, and 0.
    For instance, if a question does not pertain to the 
    respondent.
    """
    return np.nan if x in {8, 9, 0} else x

df = df_raw.applymap(convert_to_int)

df.rename(inplace=True, columns=dict(zip(
    good_columns,
    ["PartyID",
    
    "Abortion",
    "MoralRelativism",
    "NewerLifestyles",
    "MoralTolerance",
    "TraditionalFamilies",
    "GayJobDiscrimination",
    "GayAdoption",
    "GayMilitaryService",

    "NationalHealthInsurance",
    "StandardOfLiving",
    "ServicesVsSpending",

    "AffirmativeAction",
    "RacialWorkWayUp",
    "RacialGenerational",
    "RacialDeserve",
    "RacialTryHarder",
    ]
)))

non_pid_columns = list(df.columns)
non_pid_columns.remove('PartyID')
df[non_pid_columns] = df[non_pid_columns].applymap(not_informative_to_nan)

# Code so that liberal is lower numbers
df.loc[:, 'PartyID'] = df.PartyID.apply(lambda x: np.nan if x >= 7 else x)  # 7: other minor party, 8: apolitical, 9: NA

df.loc[:, 'Abortion'] = df.Abortion.apply(lambda x: np.nan if x in {7, 8, 9, 0} else -x)


df.loc[:, 'NewerLifestyles'] = df.NewerLifestyles.apply(lambda x: -x)  # Tolerance. 1: tolerance, 7: not
df.loc[:, 'TraditionalFamilies'] = df.TraditionalFamilies.apply(lambda x: -x)  # 1: moral relativism, 5: no relativism

df.loc[:, 'ServicesVsSpending'] = df.ServicesVsSpending.apply(lambda x: np.nan if x > 10 else x)
df.loc[:, 'NationalHealthInsurance'] = df.NationalHealthInsurance.apply(lambda x: np.nan if x > 10 else x)
df.loc[:, 'StandardOfLiving'] = df.StandardOfLiving.apply(lambda x: np.nan if x > 10 else x)



df.loc[:, 'ServicesVsSpending'] = df.ServicesVsSpending.apply(lambda x: -x)  # Gov't insurance?

df.loc[:, 'RacialTryHarder'] = df.RacialTryHarder.apply(lambda x: -x)  # Racial support
df.loc[:, 'RacialWorkWayUp'] = df.RacialWorkWayUp.apply(lambda x: -x)  # Systemic factors?

In [5]:
print("Variables now available: df")

Variables now available: df


In [6]:
df_rawest.V043116.value_counts()

2. Independent-Democrat (3-4-5/./5)        210
0. Strong Democrat (2/1/.)                 203
6. Strong Republican (1/1/.)               193
1. Weak Democrat (2/5-8-9/.)               179
5. Weak Republican (1/5-8-9/.)             154
4. Independent-Republican (3-4-5/./1)      138
3. Independent-Independent                 118
9. DK (8/./.)                                8
7. Other; minor party; refuses to say        5
8. Apolitical (5/./3-8-9 if apolitical)      4
Name: V043116, dtype: int64

In [7]:
df.PartyID.value_counts()

2.0    210
0.0    203
6.0    193
1.0    179
5.0    154
4.0    138
3.0    118
Name: PartyID, dtype: int64

In [8]:
df.head()

,PartyID,Abortion,MoralRelativism,NewerLifestyles,MoralTolerance,TraditionalFamilies,GayJobDiscrimination,GayAdoption,GayMilitaryService,NationalHealthInsurance,StandardOfLiving,ServicesVsSpending,AffirmativeAction,RacialWorkWayUp,RacialGenerational,RacialDeserve,RacialTryHarder
0,3.0,-2.0,2.0,-3.0,2.0,-1.0,1.0,1.0,1.0,7.0,7.0,-4.0,5.0,-1.0,5.0,2.0,-1.0
1,2.0,-2.0,2.0,-2.0,2.0,-1.0,1.0,1.0,1.0,2.0,3.0,-3.0,4.0,-2.0,1.0,2.0,-2.0
2,6.0,-2.0,5.0,-2.0,2.0,-2.0,2.0,1.0,2.0,4.0,4.0,-3.0,4.0,-2.0,4.0,3.0,-2.0
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,-2.0,5.0,-1.0,3.0,-1.0,5.0,5.0,1.0,5.0,6.0,-4.0,5.0,-1.0,5.0,2.0,-2.0


In [9]:
df.describe()

,PartyID,Abortion,MoralRelativism,NewerLifestyles,MoralTolerance,TraditionalFamilies,GayJobDiscrimination,GayAdoption,GayMilitaryService,NationalHealthInsurance,StandardOfLiving,ServicesVsSpending,AffirmativeAction,RacialWorkWayUp,RacialGenerational,RacialDeserve,RacialTryHarder
count,1195.000000,1047.000000,1064.000000,1060.000000,1062.000000,1064.000000,1025.000000,1011.000000,1023.000000,1112.000000,1103.000000,940.000000,996.000000,1061.000000,1064.000000,1057.000000,1058.000000
mean,2.872803,-2.790831,3.054511,-2.445283,2.410546,-2.008459,2.132683,3.013848,1.960899,3.657374,4.209429,-4.217021,4.094378,-2.251649,2.981203,3.310312,-2.651229
std,2.091833,1.085799,1.440196,1.277327,1.148290,1.168304,1.496727,2.000942,1.399062,1.920672,1.865394,1.359590,1.365449,1.267820,1.357750,1.217073,1.300718
min,0.000000,-4.000000,1.000000,-5.000000,1.000000,-5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-7.000000,1.000000,-5.000000,1.000000,1.000000,-5.000000
25%,1.000000,-4.000000,2.000000,-3.000000,2.000000,-2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,-5.000000,4.000000,-3.000000,2.000000,2.000000,-4.000000
50%,3.000000,-3.000000,3.000000,-2.000000,2.000000,-2.000000,1.000000,5.000000,1.000000,4.000000,4.000000,-4.000000,5.000000,-2.000000,3.000000,3.000000,-2.000000
75%,5.000000,-2.000000,4.000000,-1.000000,3.000000,-1.000000,2.000000,5.000000,2.000000,5.000000,6.000000,-3.000000,5.000000,-1.000000,4.000000,4.000000,-2.000000
max,6.000000,-1.000000,5.000000,-1.000000,5.000000,-1.000000,5.000000,5.000000,5.000000,7.000000,7.000000,-1.000000,5.000000,-1.000000,5.000000,5.000000,-1.000000


In [10]:
df.to_csv("../data/processed/2004.csv")